In [ ]:
# API Keys
NEWS_API_KEY = ""
GEMINI_API_KEY = ""

# Twitter API Keys
TWITTER_API_KEY = ""
TWITTER_API_SECRET = ""
TWITTER_ACCESS_TOKEN = ""
TWITTER_ACCESS_TOKEN_SECRET = ""

#Please enter your own API keys

In [ ]:
import requests
import google.generativeai as genai
from requests_oauthlib import OAuth1Session
import json
import hashlib
from datetime import datetime, timedelta

# API Endpoints
NEWS_API_URL = "https://newsapi.org/v2/everything"
TWITTER_API_URL = "https://api.twitter.com/2/tweets"

POSTED_ARTICLES_FILE = "posted_articles.json"
MAX_TWEET_LENGTH = 280
MAX_ATTEMPTS = 3

# Configure Gemini
genai.configure(api_key=GEMINI_API_KEY)
model = genai.GenerativeModel("gemini-pro")


# Agent 1: NewsFetcherAgent
class NewsFetcherAgent:
    @staticmethod
    def fetch_one_news():
        """Fetch one AI/ML/Tech-related news article."""
        params = {
            "q": "(artificial intelligence OR machine learning OR deep learning) AND (technology OR tech)",
            "language": "en",
            "sortBy": "popularity",
            "apiKey": NEWS_API_KEY,
            "pageSize": 1,  # Fetch only 1 article
        }
        try:
            response = requests.get(NEWS_API_URL, params=params)
            response.raise_for_status()
            articles = response.json().get("articles", [])
            return articles[0] if articles else None
        except requests.exceptions.RequestException as e:
            print(f"Error fetching news: {e}")
            return None


# Agent 2: ContentSummarizerAgent
class ContentSummarizerAgent:
    @staticmethod
    def summarize_content(article):
        """Summarize the news article using Gemini API."""
        try:
            prompt = f"""
            Summarize this AI/ML/Tech article concisely in 1-2 sentences:
            Title: {article['title']}
            Content: {article['content'][:1000]}
            Focus on the technical aspects and innovations.
            """
            response = model.generate_content(prompt)
            return response.text.strip()
        except Exception as e:
            print(f"Error summarizing content: {e}")
            return None


# Agent 3: TweetCraftingAgent
class TweetCraftingAgent:
    @staticmethod
    def craft_tweet(summary, url, attempt=1):
        """Craft a concise and engaging tweet."""
        try:
            url_length = len(url) + 1
            hashtags = " #AI #ML #TechNews"
            available_chars = MAX_TWEET_LENGTH - (url_length + len(hashtags))

            prompt = f"""
            Create an engaging tweet for this AI/ML/Tech news summary.
            Summary: {summary}
            Constraints:
            - Max {available_chars} characters (excluding URL and hashtags)
            - Use 1-2 emojis
            - Concise and engaging.
            Attempt {attempt} of {MAX_ATTEMPTS}.
            """
            response = model.generate_content(prompt)
            tweet_content = response.text.strip()

            full_tweet = f"{tweet_content} {url}{hashtags}"
            if len(full_tweet) > MAX_TWEET_LENGTH and attempt < MAX_ATTEMPTS:
                return TweetCraftingAgent.craft_tweet(summary, url, attempt + 1)
            return full_tweet
        except Exception as e:
            print(f"Error crafting tweet: {e}")
            return None


# Agent 4: TwitterPostingAgent
class TwitterPostingAgent:
    @staticmethod
    def post_tweet(tweet):
        """Post the tweet using Twitter API."""
        try:
            oauth = OAuth1Session(
                TWITTER_API_KEY,
                client_secret=TWITTER_API_SECRET,
                resource_owner_key=TWITTER_ACCESS_TOKEN,
                resource_owner_secret=TWITTER_ACCESS_TOKEN_SECRET,
            )
            response = oauth.post(TWITTER_API_URL, json={"text": tweet})
            if response.status_code != 201:
                raise Exception(f"Error: {response.status_code} {response.text}")
            print("Tweet posted successfully!")
            return True
        except Exception as e:
            print(f"Error posting tweet: {e}")
            return False


# Agent 5: ArticleManagerAgent
class ArticleManagerAgent:
    @staticmethod
    def load_posted_articles():
        """Load hashes of posted articles."""
        try:
            with open(POSTED_ARTICLES_FILE, "r") as f:
                return json.load(f)
        except FileNotFoundError:
            return {"articles": []}

    @staticmethod
    def save_posted_articles(posted_articles):
        """Save posted article hashes."""
        with open(POSTED_ARTICLES_FILE, "w") as f:
            json.dump(posted_articles, f)

    @staticmethod
    def is_article_posted(article, posted_articles):
        """Check if the article has already been posted."""
        content_hash = hashlib.md5(f"{article['title']}{article['url']}".encode()).hexdigest()
        return content_hash in posted_articles["articles"]

    @staticmethod
    def add_article(article, posted_articles):
        """Mark the article as posted."""
        content_hash = hashlib.md5(f"{article['title']}{article['url']}".encode()).hexdigest()
        posted_articles["articles"].append(content_hash)
        ArticleManagerAgent.save_posted_articles(posted_articles)


# Main Function: Orchestrating Agents
def main():
    """Fetch, summarize, and post one tweet."""
    # Load posted articles
    posted_articles = ArticleManagerAgent.load_posted_articles()

    # Fetch one news article
    article = NewsFetcherAgent.fetch_one_news()
    if not article:
        print("No articles found.")
        return

    # Skip if already posted
    if ArticleManagerAgent.is_article_posted(article, posted_articles):
        print("Article already posted. Exiting.")
        return

    # Summarize content
    summary = ContentSummarizerAgent.summarize_content(article)
    if not summary:
        print("Failed to summarize article. Exiting.")
        return

    # Craft tweet
    tweet = TweetCraftingAgent.craft_tweet(summary, article["url"])
    if not tweet:
        print("Failed to craft tweet. Exiting.")
        return
        
    # Print tweet to console
    print("\nCrafted Tweet:")
    print(tweet)
    
    # Post tweet
    if TwitterPostingAgent.post_tweet(tweet):
        ArticleManagerAgent.add_article(article, posted_articles)


if __name__ == "__main__":
    main()